In [11]:
import json
import spacy
import boto3

def get_model(s3_path):
    """
    Loads the model from s3 bucket and saved in folder /tmp/model
    """
    s3_resource = boto3.resource("s3",)
    dir_path = s3_path + '/models/skill_ner_model/'
    s3_resource.meta.client.download_file(s3_path, '/models/skill_ner_model/', '/tmp/model/')

    
    
def download_dir(client, resource, dist, local='/tmp', bucket='s3bucket'):
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Delimiter='/', Prefix=dist):
        if result.get('CommonPrefixes') is not None:
            for subdir in result.get('CommonPrefixes'):
                download_dir(client, resource, subdir.get('Prefix'), local, bucket)
        if result.get('Contents') is not None:
            for file in result.get('Contents'):
                if not os.path.exists(os.path.dirname(local + os.sep + file.get('Key'))):
                     os.makedirs(os.path.dirname(local + os.sep + file.get('Key')))
                resource.meta.client.download_file(bucket, file.get('Key'), local + os.sep + file.get('Key'))


def lambda_handler(event, context):
    """
    Main Function zum ausführen der restlichen Codes und laden der benötigten Daten aus den Datenbanken
    Schreibt auch die Ergebnisse zurück in die DB
    """

    client = boto3.client('s3')
    resource = boto3.resource('s3')
    bucket_name =  "job-app-data-bucket"
    get_model(bucket_name)
    object_key = "skill_ner_model"  # replace object key
    if (os.path.isdir("/tmp/skill_ner_model")==False):
        download_dir(client, resource, 'skill_ner_model', '/tmp',bucket_name)
    spacy.util.set_data_path('/tmp')
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


In [ ]:
def download_dir(client, resource, dist, local='/tmp', bucket='s3bucket'):
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Delimiter='/', Prefix=dist):
        if result.get('CommonPrefixes') is not None:
            for subdir in result.get('CommonPrefixes'):
                download_dir(client, resource, subdir.get('Prefix'), local, bucket)
        if result.get('Contents') is not None:
            for file in result.get('Contents'):
                if not os.path.exists(os.path.dirname(local + os.sep + file.get('Key'))):
                     os.makedirs(os.path.dirname(local + os.sep + file.get('Key')))
                resource.meta.client.download_file(bucket, file.get('Key'), local + os.sep + file.get('Key'))

In [ ]:
def handler(event, context):
    client = boto3.client('s3')
    resource = boto3.resource('s3')
    if (os.path.isdir("/tmp/skill_ner_model")==False):
        download_dir(client, resource, 'skill_ner_model', '/tmp',bucket_name)
    spacy.util.set_data_path('/tmp')
    nlp = spacy.load('/tmp/en_core_web_sm/en_core_web_sm-2.0.0')
    doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')
    for token in doc:
        print(token.text, token.pos_, token.dep_)
    return 'finished'

In [12]:
get_model("job-app-data-bucket")

NoCredentialsError: Unable to locate credentials